In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot
import plot as plot
import flux as flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm

In [ ]:
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'

# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS
alpha = plot.ALPHA

# Read in Snapshots

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    # shot[ii].load_shock_variables()
    snaps[ii].load_fluxes()

match scale axes

In [ ]:
def plot_4panel_spectra(snaps, labels, xx=freqs/10**9, 
    row_text = ['no cooling', 'with cooling'], col_text = ['1000 au', '25000 au'],
    xx_text=0.99, yy_text=0.025, match='rows',
    xlabel = r'Frequency, $\nu$ [GHz]', ylabel = r'Flux $S_\nu$ [mJy]',
    markers = np.array(['v', 's', 'x', '*', 'd', 'P']), colors = plot.COLORS,
    ):


    fig, axs = plot.figax(
        nrows=2, ncols=2, figsize=(11,7), sharex=True)

    if match=='rows':
        axs[1,0].sharey(axs[0,0])
        axs[1,1].sharey(axs[0,1])
    elif match=='cols': 
        axs[0,1].sharey(axs[0,0])
        axs[1,1].sharey(axs[1,0])
    for ax in axs[1,:]:
        ax.set_xlabel(xlabel)
    for ax in axs[:,0]:
        ax.set_ylabel(ylabel)

    for ii, row in enumerate(axs):
        for jj, ax in enumerate(row):
            ax.text(xx_text, yy_text, (col_text[jj]+'\n'+row_text[ii]),
                    weight='bold', horizontalalignment='right', transform=ax.transAxes,)
            
    for ss, snap in enumerate(snaps):
        y00 = snap.ScaleFluxes[0,:,1]
        y01 = snap.ScaleFluxes[0,:,-1]
        y10 = snap.ScaleFluxes_ratio[:,1]
        y11 = snap.ScaleFluxes_ratio[:,-1]
        yy = np.array([y00, y01, y10, y11])


        for ii, ax in enumerate(axs.flatten()):
            ax.plot(xx, yy[ii], color=colors[ss], label=labels[ss], marker=markers[ss], alpha=plot.ALPHA)

    # if ax0_text is not None and ax1_text is not None:
    #     axtext = np.array([ax0_text, ax0_text, ax1_text, ax1_text])
    #     for ii, ax in enumerate(axs.flatten()):
    #         ax.text(xx_text, yy_text, axtext[ii], transform=ax.transAxes,
    #                 weight='bold', horizontalalignment='right')

    fig.tight_layout()
    return fig


match scales (cols)

In [ ]:

fig = plot_4panel_spectra(snaps[1:], labels=masss[1:], match='cols')
ax = fig.axes[0]
ax.legend(loc='upper left')
fig.tight_layout()
fig.savefig(VICO_loc+'/figures/spectra/spectra_4panel_matchscale.png', dpi=300)

match cooling (rows)

In [ ]:
fig = plot_4panel_spectra(snaps[1:], labels=masss[1:], match='rows')
ax = fig.axes[0]
ax.legend(loc='upper left')
fig.tight_layout()
fig.savefig(VICO_loc+'/figures/spectra/spectra_4panel_matchcooling.png', dpi=300)

## combined_spectra

In [ ]:
def plot_combined_spectra(snaps, labels, xx=freqs/10**9, 
    # row_text = ['no cooling', 'with cooling'],
    col_text = ['1000 au', '25000 au'],
    xx_text=0.99, yy_text=0.025,
    xlabel = r'Frequency, $\nu$ [GHz]', ylabel = r'Flux $S_\nu$ [mJy]',
    markers = np.array(['v', 's', 'x', '*', 'd', 'P']), colors = plot.COLORS,
    ylim2 = (10**-1.8, 10**2.9)
    ):


    fig, axs = plot.figax(
        nrows=1, ncols=2, figsize=(11,4), sharex=True, xlabel=xlabel)

    axs[0].set_ylabel(ylabel)
    for ii, ax in enumerate(axs):
        ax.text(xx_text, yy_text, col_text[ii],
                weight='bold', horizontalalignment='right', transform=ax.transAxes,)
            
    for ss, snap in enumerate(snaps):
        y1 = [snap.ScaleFluxes[0,:,1], snap.ScaleFluxes[0,:,-1]]
        y2 = [snap.ScaleFluxes_ratio[:,1], snap.ScaleFluxes_ratio[:,-1]]

        for ii, ax in enumerate(axs.flatten()):
            ax.plot(xx, y1[ii], color=colors[ss], label=None, marker=markers[ss], alpha=plot.ALPHA, linestyle='--')
            ax.plot(xx, y2[ii], color=colors[ss], label=labels[ss], marker=markers[ss], alpha=plot.ALPHA, linestyle='-')

    axs[0].legend(loc='upper left')
    axs[1].set_ylim(ylim2)
    # if ax0_text is not None and ax1_text is not None:
    #     axtext = np.array([ax0_text, ax0_text, ax1_text, ax1_text])
    #     for ii, ax in enumerate(axs.flatten()):
    #         ax.text(xx_text, yy_text, axtext[ii], transform=ax.transAxes,
    #                 weight='bold', horizontalalignment='right')

    fig.tight_layout()
    return fig

fig = plot_combined_spectra(snaps[1:], labels=masss[1:])
fig.savefig(VICO_loc+'/figures/spectra/spectra_combined.png', dpi=300)


# All-Res

In [ ]:
rnums = np.array([9, 21, 39, 54])
ryears = years[1:len(rnums)+1]
rmasss = masss[1:len(rnums)+1]
snaps_lr, snaps_mr, snaps_hr = flux.prep_spec_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr',])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr',])
mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

In [ ]:
print(snaps_lr[0].__dict__.keys())
print(scales)
print(f"{snaps_lr[0].ScaleFluxes.shape=}")
print(f"{snaps_lr[0].ScaleFluxes_ratio.shape=}")

In [ ]:
def plot_allres_spectra(snaps_lr, snaps_mr, snapr_hr, labels, xx=freqs/10**9, 
    row_text = ['no cooling', 'with cooling'], col_text = ['1000 au', '25000 au'],
    xx_text=0.99, yy_text=0.025, match_rows=False, match='rows',
    xlabel = r'Frequency, $\nu$ [GHz]', ylabel = r'Flux $S_\nu$ [mJy]',
    markers = np.array(['v', 's', 'x', '*', 'd', 'P']), colors = plot.COLORS,
    ):


    fig, axs = plot.figax(
        nrows=2, ncols=2, figsize=(11,7), sharex=True)
    
    if match=='rows':
        axs[1,0].sharey(axs[0,0])
        axs[1,1].sharey(axs[0,1])
    elif match=='cols': 
        axs[0,1].sharey(axs[0,0])
        axs[1,1].sharey(axs[1,0])

    for ax in axs[1,:]:
        ax.set_xlabel(xlabel)
    for ax in axs[:,0]:
        ax.set_ylabel(ylabel)

    for ii, row in enumerate(axs):
        for jj, ax in enumerate(row):
            ax.text(xx_text, yy_text, (col_text[jj]+'\n'+row_text[ii]),
                    weight='bold', horizontalalignment='right', transform=ax.transAxes,)
    linestyles=[':', '--', '-']
    markstyles=['x', '+', 'o']
    for nn, snaps in enumerate([snaps_lr, snaps_mr, snaps_hr]):        
        ls = linestyles[nn]
        ms = markstyles[nn]
        for ss, snap in enumerate(snaps):
            y00 = snap.ScaleFluxes[0,:,1]
            y01 = snap.ScaleFluxes[0,:,-1]
            y10 = snap.ScaleFluxes_ratio[:,1]
            y11 = snap.ScaleFluxes_ratio[:,-1]
            yy = np.array([y00, y01, y10, y11])
            for ii, ax in enumerate(axs.flatten()):
                label=labels[ss] if nn==2 else None
                ax.plot(xx, yy[ii], color=colors[ss], label=label, marker=ms, ls=ls, alpha=plot.ALPHA)

    # if ax0_text is not None and ax1_text is not None:
    #     axtext = np.array([ax0_text, ax0_text, ax1_text, ax1_text])
    #     for ii, ax in enumerate(axs.flatten()):
    #         ax.text(xx_text, yy_text, axtext[ii], transform=ax.transAxes,
    #                 weight='bold', horizontalalignment='right')

    fig.tight_layout()
    return fig

In [ ]:

fig = plot_allres_spectra(snaps_lr, snaps_mr, snaps_hr, labels=rmasss, match='rows')
ax = fig.axes[0]
ax.legend(loc='upper left')
fig.tight_layout()
fig.savefig(VICO_loc+'/figures/spectra/spectra_allres_matchcooling.png', dpi=300)

In [ ]:
fig = plot_allres_spectra(snaps_lr, snaps_mr, snaps_hr, labels=rmasss, match='cols')
ax = fig.axes[0]
ax.legend(loc='upper left')
fig.tight_layout()
fig.savefig(VICO_loc+'/figures/spectra/spectra_allres_matchscale.png', dpi=300)